# RNN model of URIs

The Python code from this notebook has now been developed into two scripts
<ol>
<li> <tt>train_rnn.py</tt>
<code>
Usage: python  ./python/train_rnn.py infile [options]
Options are:
        -o outfile 
        -datapath [./sdata]
        -modelpath [./models]
        -unroll [20]
        -step [3]
        -dropout [0.1]
        -niters [10]
        -arch (hidden layer sizes) [16]
</code>
<li> <tt>make_synthetic.py</tt>
<code>
Usage: python  ./python/make_synthetic.py infile [options]
Options are:
        -n length of output [1000]
        -init        [newlines]
        -modelpath        [./models]</code>
</ol>

# Training a model

In [38]:
run -i train_rnn\
    smallstring.gz\
    -datapath ../sdata\
    -modelpath ../models\
    -arch 16,16

Checking hardware ...
[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18166164399265663188
]
Model architecture:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 20, 16)            1600      
_________________________________________________________________
dropout_4 (Dropout)          (None, 20, 16)            0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 16)                2112      
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               4352      
_________________________________________________________________
activation_7 (Activation)    (None, 256)               0  

# Synthetic domain string from model

In [41]:
run -i make_synthetic\
    model_from_big_domain_string_1.gz_arch_8_16_unroll_20_step_3_dropout_0.1_iter_7.npy\
    -modelpath ../models\
    -n 500\
    -init '/tag/маша и медведь мультфильм'   

/tag/маша и медведь мультфильм�9%la%f8%07%-f967/%b1%db%daa-%d8%81%84%b1%80%88%86%qo%20%11%.2%850S2bAaECy/%86%B3%Ae%D8%BH%e4%Bt%ff%B4%D6%8d%B0%8c/mr.html
/tax.gxe
/lote-su-ofd-ju-tro/nadel/chx/346-

welNmm.hpm-h-osdin/euts/evem/
/wpo1GC_81.cog
/blrig-p-do/r3659/nor-dane454vrl-0132/precs/eisides/larykde
/fatebvo/moch.jpg
/q1%aE%d8%70%Ad%90%2%L0%E6%b9%88%D7%3DEuIcs/10043800.jpg
/sklcy-iog-wourtrre-make.html
xeg
/ve-naim-fxo-kamstp-cockal
/sitemt-172d397/2
/iceqn-wiarnemarus/floonc
_oages/comsgrzaey-noms-wcder/gan-rianer/
/2017/
